# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


Definimos la estructura como un vector, esto nos va a permitir cambiar el tamaño de manera dinámica.

In [1]:
doc"""
Definición de polinomios de Taylor, donde coef es el arreglo que contiene al polinomio de taylor 
...
"""
type Taylor{T<:Number}
    coef::Array{T}
end

Taylor

Creamos una función constructora.

In [2]:
doc"""
función constructora, inicia una variable de tipo Taylor de grado 1.
"""

function Taylor(a::Number)
    return Taylor([a])
end

Taylor

Para poder operar con este nuevo tipo de variables, necesitamos que sean consistentes en longitud, para esto utilizamos la función gradomax() y la función prom().

In [3]:
doc"""
Función que regresa la longitud maxima entre dos polinomios de Taylor. Al ingresar solo un polinomio, regresa el grado de este.
El grado de los polinomios esta dado por gradomax(x) - 1.
"""
function gradomax(a::Taylor, b::Taylor = a)
    return max(length(a.coef), length(b.coef))
end

function gradomax(a::Taylor)
    return length(a.coef)
end

gradomax (generic function with 2 methods)

In [4]:
doc"""
Función que promueve el grado del polinomio a a al del polinomio b.
Si en lugar de un segundo polinomio se le da un entero, esta promueve el primer polinomio al grado que se le dio.
"""

function prom(a::Taylor, b::Taylor)
    return Taylor([a.coef; fill(0, gradomax(a,b) - gradomax(a))])
end

function prom(a::Taylor, N::Integer)
    return Taylor([a.coef; fill(0, N - gradomax(a))])
end

prom (generic function with 2 methods)

In [5]:
doc"""
Función que utiliza metaprogramación para evaluar un polinomio de Taylor
"""

function evaluacion(a::Taylor, x0::Number)
    polinomio = :(0)  ## Ok, usemos algo de meta programacion
    for k in 1:gradomax(a)
        polinomio = :($polinomio + $a.coef[$k] * $x0 ^ $(k - 1))
    end      # Esta conformado por las iteraciones anteriores + el coeficiente por x0 ^ k-1
    return eval(polinomio)
end

evaluacion

Utilizando las reglas determinadas en clase

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

Definimos las siguientes operaciones.

In [6]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(a::Taylor, b::Taylor) = Taylor(prom(a, gradomax(a, b)).coef + prom(b, gradomax(a, b)).coef)
+(a::Taylor, b::Number) = a + Taylor(b)
+(a::Number, b::Taylor) = b + Taylor(a)
+(a::Taylor) = Taylor(a.coef)
 ## Tanto la suma como la resta de polinomios promueven el resultado a el grado mayor entre los sumandos.
-(a::Taylor, b::Taylor) = Taylor(prom(a, gradomax(a, b)).coef - prom(b, gradomax(a, b)).coef)
-(a::Taylor, b::Number) = a - Taylor(b)
-(a::Number, b::Taylor) = Taylor(a) - b
-(a::Taylor) = Taylor(-a.coef)

function *(a::Taylor, b::Taylor)
    n = gradomax(a, b)
    A = prom(a,n)
    B = prom(b,n)
    producto = Taylor(zeros(n)) ## Inicializamos un polinomio de Taylor de grado n.
    
    for k = 0:(n - 1)
        suma = 0
        for j = 0:k
            suma += A.coef[j + 1] * B.coef[k - j + 1]
        end
        producto.coef[k + 1] = suma

    end
    
    return producto
end
*(a::Taylor, b::Number) = Taylor(b * a.coef)
*(a::Number, b::Taylor) = Taylor(a * b.coef)

function /(a::Taylor, b::Taylor)
    n = gradomax(a, b)
    A = prom(a, n)
    B = prom(b, n)

    div = Taylor(zeros(n))
    ## Checamos por coeficientes nulos
    p = 1
    while b.coef[p] == 0
        p += 1;
    end

    div.coef[1] = A.coef[p] / B.coef[p] ## Agregamos el primer elemento

    for k = (p + 1):n
        suma = 0
        
        for j = 0:(k - 1)
            suma += div.coef[j + 1] * B.coef[k - j]
        end

        div.coef[k - p + 1] = (A.coef[k] - suma) / B.coef[p]
    end
    
    return div
end

/(a::Taylor, k::Number) = Taylor(a.coef / k)
/(k::Number, a::Taylor) = Taylor(k) / a

## La función == promueve los polinomios y despues los compara, asi podemos ver si los coeficientes son iguales
## sin considerar la diferencia en el grado, asi Taylor([1,1]) == Taylor([1,1,0])
==(a::Taylor, b::Taylor) = prom(a, b).coef == prom(b, a).coef

== (generic function with 119 methods)

In [7]:
x = Taylor([1.0, 1.0])

Taylor{Float64}([1.0,1.0])

In [8]:
x2 = x * x

Taylor{Float64}([1.0,2.0])

Se agregan archivos runtest_Taylor.jl y Taylor.jl

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

Planteando la función 

$$ E(x) = \exp{(g(x))} $$
$$ \frac{dE}{dx}(x) = \frac{dg}{dx}(x) \exp{(g(x))} = \frac{dg}{dx}(x) E(x)$$

Ahora si expresamos a $E(x)$ y a $\frac{dg}{dx}(x)$ como polinomios de Taylor tenemos que

$$ E(x) = \sum_{k = 0}^{\infty} E_{[k]}(x - x_0)^k$$
$$ \frac{dE}{dx}(x) = \sum_{k = 0}^{\infty} kE_{[k]}(x - x_0)^{(k - 1)}$$
$$ \frac{dg}{dx}(x) = \sum_{k = 0}^{\infty} kg_{[k]}(x - x_0)^{(k - 1)}$$

Por lo tanto 

$$ \sum_{k = 0}^{\infty} kE_{[k]}(x - x_0)^{(k - 1)} = (\sum_{k = 0}^{\infty} E_{[k]}(x - x_0)^k) (\sum_{k = 0}^{\infty} kg_{[k]}(x - x_0)^{(k - 1)})$$

donde el lado derecho de la ecuacion es un producto de polinomios, por lo tanto podemos utilizar la regla de producto antes vista. Jugando con los indices de las sumas podemos llegar a

$$ \sum_{k = 0}^{\infty} kE_{[k]}(x - x_0)^{(k - 1)} = \sum_{k = 0}^{\infty} (\sum_{j = 0}^{k} (k - j)g_{[k -j]} E_{[j]})(x - x_0)^{(k - 1)}$$

por lo tanto

$$ E_{[k]} = \frac{1}{k} (\sum_{j = 0}^{k} (k - j)g_{[k -j]} E_{[j]}) $$

In [ ]:
import Base: exp

function exp(a::Taylor)
    n = gradomax(a)
    
    expo = Taylor(exp(a.coef[1]))
    expo = prom(expo, n)
    
    for k = 2:n
        suma = 0
        for j = 1:k
            suma += (k - j) * a.coef[k - j + 1] * expo.coef[j]
        end
        expo.coef[k] = suma * (1 / (k - 1))
    end
    
    return expo
end

In [ ]:
using PyPlot

In [ ]:
x = linspace(-1.5, 1.5, 100)
y = zeros(x)
x_ = Taylor([0.0, 0.0, -1.0])

for k = gradomax(x_):10
    x_ = prom(x_, k)
    ec = exp(x_)
    for (i, j) = enumerate(x)
        y[i] = evaluacion(ec, j)
    end
    plot(x, y, label = "k = $(k)")
end
plot(x, exp(-x.*x), label = "analitica")
legend()

Para el logaritmo realizamos el mismo procedimiento

$$ L(x) = log(g(x)) $$
$$ \frac{dL}{dx}(x) =  \frac{1}{g(x)} \frac{dg}{dx}(x) $$

Aqui si vemos a $ \frac{1}{g(x)} \frac{dg}{dx}(x) $ como
$$ \frac{g'}{g} = \sum_{k = 0}^{\infty} (\frac{g'}{g})_{[k]} (x - x_0)^k$$

Entonces tenemos

$$ \frac{dL}{dx}(x) = \sum_{k = 0}^{\infty} kL (x - x_0) ^{k - 1} = \sum_{k = 0}^{\infty} (\frac{g'}{g})_{[k]} (x - x_0)^k$$

por lo tanto

$$ L_{[k]} = \frac{1}{k} (\frac{g'}{g})_{[k]}$$

In [ ]:
import Base: log

function log(a::Taylor)
    n = gradomax(a)

    loga = Taylor(log(a.coef[1]))
    loga = prom(loga, n)

    p = 1
    while a.coef[p] == 0
        p += 1
    end
    
    for k = (p + 1):n
        suma = 0
        
        for j = (p + 1):k
            suma += (j - 1) * loga.coef[j] * a.coef[k - j + 1]
        end
        loga.coef[k] = (1 / a.coef[p]) * (a.coef[k] - suma / (k - p))
    end
    
    return loga
end

In [ ]:
x = linspace(1, 100, 100)
y = zeros(x)
x_ = Taylor([0.0, 1.0])

for k = 2:5
    x_ = prom(x_, k)
    ec = log(x_)
    for (i, j) = enumerate(x)
        y[i] = evaluacion(ec, j)
    end
    plot(x, y, label = "k = $(k)")
end
# plot(x, log(x), label = "analitica")
legend()

In [ ]:
import Base: ^

function ^(a::Taylor, n::Integer)
    if n != 0
        pot = a
        for i = 2:n
            pot *= a
        end
        return pot
    else
        return Taylor([1])
    end
end



^(a::Taylor, N::Number) = exp(N * log(a))